# Validate Results
Yue, Maca, Rukhshan

Our best random forsest model seems to classfiy less active accounts as bots. Although the precision score is very high, we want to check whether they tend to misclassify less active human accounts as bots. 

## 1. Prepare Dataset

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df_time = pd.read_csv("../Exploring-Features/created_time_accounts_test.csv")
df_time.head()

,index,bot_x,label_predicted,created_at,probe_timestamp
0,21228,1,1,Thu Oct 18 23:42:42 2018,Wed Oct 24 18:47:39 2018
1,48490,0,0,Tue Jun 19 20:13:50 2012,Tue Nov 06 11:11:27 2018
2,29369,1,1,Sat Oct 27 08:55:12 2018,Sun Oct 28 01:17:31 2018
3,21131,1,1,Tue Oct 16 09:54:02 2018,Tue Oct 16 10:02:32 2018
4,52764,0,0,Tue May 13 06:15:17 +0000 2014,Mon Jan 21 01:00:06 +0000 2019


In [2]:
df_consolidated = pd.read_csv("../data_consolidation/consolidated_version2.csv")
df_consolidated['index_original'] = df_consolidated.index
df_subset =  df_consolidated[[ "index_original",'bot', 'created_at', 'probe_timestamp']].copy()
df_subset.head()

,index_original,bot,created_at,probe_timestamp
0,0,human,Sun Feb 15 14:56:36 +0000 2015,Thu May 16 13:57:12 +0000 2019
1,1,bot,Fri Oct 14 08:00:55 +0000 2011,Tue Apr 16 13:51:17 +0000 2019
2,2,human,Sat Dec 26 13:55:00 +0000 2015,Sun May 12 17:01:48 +0000 2019
3,3,human,Sun May 22 14:53:38 +0000 2016,Fri May 17 16:59:10 +0000 2019
4,4,bot,Sun Jun 24 20:09:50 +0000 2018,Wed Feb 13 01:56:38 +0000 2019


In [3]:
# Join the two datasets together. 
merged_df = pd.merge(left=df_time, right=df_subset, how='left', left_on='index', right_on='index_original')
merged_df.head()

,index,bot_x,label_predicted,created_at_x,probe_timestamp_x,index_original,bot,created_at_y,probe_timestamp_y
0,21228,1,1,Thu Oct 18 23:42:42 2018,Wed Oct 24 18:47:39 2018,21228,bot,Thu Oct 18 23:42:42 2018,Wed Oct 24 18:47:39 2018
1,48490,0,0,Tue Jun 19 20:13:50 2012,Tue Nov 06 11:11:27 2018,48490,human,Tue Jun 19 20:13:50 2012,Tue Nov 06 11:11:27 2018
2,29369,1,1,Sat Oct 27 08:55:12 2018,Sun Oct 28 01:17:31 2018,29369,bot,Sat Oct 27 08:55:12 2018,Sun Oct 28 01:17:31 2018
3,21131,1,1,Tue Oct 16 09:54:02 2018,Tue Oct 16 10:02:32 2018,21131,bot,Tue Oct 16 09:54:02 2018,Tue Oct 16 10:02:32 2018
4,52764,0,0,Tue May 13 06:15:17 +0000 2014,Mon Jan 21 01:00:06 +0000 2019,52764,human,Tue May 13 06:15:17 +0000 2014,Mon Jan 21 01:00:06 +0000 2019


In [8]:
#Convert the time columns into datetime objects
merged_df["created_at_x"] = pd.to_datetime(merged_df["created_at_x"],utc=True)
merged_df["created_at_x"] = merged_df.created_at_x.dt.tz_convert('MST')
merged_df["probe_timestamp_x"] = pd.to_datetime(merged_df["probe_timestamp_x"],utc=True)
merged_df["probe_timestamp_x"] = merged_df.probe_timestamp_x.dt.tz_convert('MST')

merged_df.head()

,index,bot_x,label_predicted,created_at_x,probe_timestamp_x,index_original,bot,created_at_y,probe_timestamp_y
0,21228,1,1,2018-10-18 16:42:42-07:00,2018-10-24 11:47:39-07:00,21228,bot,Thu Oct 18 23:42:42 2018,Wed Oct 24 18:47:39 2018
1,48490,0,0,2012-06-19 13:13:50-07:00,2018-11-06 04:11:27-07:00,48490,human,Tue Jun 19 20:13:50 2012,Tue Nov 06 11:11:27 2018
2,29369,1,1,2018-10-27 01:55:12-07:00,2018-10-27 18:17:31-07:00,29369,bot,Sat Oct 27 08:55:12 2018,Sun Oct 28 01:17:31 2018
3,21131,1,1,2018-10-16 02:54:02-07:00,2018-10-16 03:02:32-07:00,21131,bot,Tue Oct 16 09:54:02 2018,Tue Oct 16 10:02:32 2018
4,52764,0,0,2014-05-12 23:15:17-07:00,2019-01-20 18:00:06-07:00,52764,human,Tue May 13 06:15:17 +0000 2014,Mon Jan 21 01:00:06 +0000 2019


In [10]:
# Calculate time difference
merged_df['time_diff'] = (merged_df['probe_timestamp_x']-merged_df['created_at_x']).astype('timedelta64[D]')
merged_df.head()

,index,bot_x,label_predicted,created_at_x,probe_timestamp_x,index_original,bot,created_at_y,probe_timestamp_y,time_diff
0,21228,1,1,2018-10-18 16:42:42-07:00,2018-10-24 11:47:39-07:00,21228,bot,Thu Oct 18 23:42:42 2018,Wed Oct 24 18:47:39 2018,5.0
1,48490,0,0,2012-06-19 13:13:50-07:00,2018-11-06 04:11:27-07:00,48490,human,Tue Jun 19 20:13:50 2012,Tue Nov 06 11:11:27 2018,2330.0
2,29369,1,1,2018-10-27 01:55:12-07:00,2018-10-27 18:17:31-07:00,29369,bot,Sat Oct 27 08:55:12 2018,Sun Oct 28 01:17:31 2018,0.0
3,21131,1,1,2018-10-16 02:54:02-07:00,2018-10-16 03:02:32-07:00,21131,bot,Tue Oct 16 09:54:02 2018,Tue Oct 16 10:02:32 2018,0.0
4,52764,0,0,2014-05-12 23:15:17-07:00,2019-01-20 18:00:06-07:00,52764,human,Tue May 13 06:15:17 +0000 2014,Mon Jan 21 01:00:06 +0000 2019,1713.0


In [11]:
# Convert bot column to a binary column with value 1 and 0
merged_df.loc[merged_df['bot']=="bot",'bot_actual'] = 1
merged_df.loc[merged_df['bot']=="human",'bot_actual'] = 0
merged_df.head()

,index,bot_x,label_predicted,created_at_x,probe_timestamp_x,index_original,bot,created_at_y,probe_timestamp_y,time_diff,bot_actual
0,21228,1,1,2018-10-18 16:42:42-07:00,2018-10-24 11:47:39-07:00,21228,bot,Thu Oct 18 23:42:42 2018,Wed Oct 24 18:47:39 2018,5.0,1.0
1,48490,0,0,2012-06-19 13:13:50-07:00,2018-11-06 04:11:27-07:00,48490,human,Tue Jun 19 20:13:50 2012,Tue Nov 06 11:11:27 2018,2330.0,0.0
2,29369,1,1,2018-10-27 01:55:12-07:00,2018-10-27 18:17:31-07:00,29369,bot,Sat Oct 27 08:55:12 2018,Sun Oct 28 01:17:31 2018,0.0,1.0
3,21131,1,1,2018-10-16 02:54:02-07:00,2018-10-16 03:02:32-07:00,21131,bot,Tue Oct 16 09:54:02 2018,Tue Oct 16 10:02:32 2018,0.0,1.0
4,52764,0,0,2014-05-12 23:15:17-07:00,2019-01-20 18:00:06-07:00,52764,human,Tue May 13 06:15:17 +0000 2014,Mon Jan 21 01:00:06 +0000 2019,1713.0,0.0


In [30]:
#Select accounts only created for a few days
df_five_days = merged_df[merged_df["time_diff"]<=90]
df_five_days.head()

,index,bot_x,label_predicted,created_at_x,probe_timestamp_x,index_original,bot,created_at_y,probe_timestamp_y,time_diff,bot_actual
0,21228,1,1,2018-10-18 16:42:42-07:00,2018-10-24 11:47:39-07:00,21228,bot,Thu Oct 18 23:42:42 2018,Wed Oct 24 18:47:39 2018,5.0,1.0
2,29369,1,1,2018-10-27 01:55:12-07:00,2018-10-27 18:17:31-07:00,29369,bot,Sat Oct 27 08:55:12 2018,Sun Oct 28 01:17:31 2018,0.0,1.0
3,21131,1,1,2018-10-16 02:54:02-07:00,2018-10-16 03:02:32-07:00,21131,bot,Tue Oct 16 09:54:02 2018,Tue Oct 16 10:02:32 2018,0.0,1.0
5,37484,1,1,2018-10-29 08:23:03-07:00,2018-11-07 14:28:36-07:00,37484,bot,Mon Oct 29 15:23:03 2018,Wed Nov 07 21:28:36 2018,9.0,1.0
10,3294,1,1,2018-12-09 09:10:24-07:00,2018-12-09 09:38:46-07:00,3294,bot,Sun Dec 09 16:10:24 2018,Sun Dec 09 16:38:46 2018,0.0,1.0


## 2. Analyze

In [31]:
df_matrix = pd.DataFrame(df_five_days, columns=['bot_actual','label_predicted'])
confusion_matrix = pd.crosstab(df_matrix['bot_actual'], df_matrix['label_predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted   0     1
Actual             
0.0        10    38
1.0        67  7078
